# Notebook to calculate CW-SSIM, PSNR, FID, and other Pytorch metrics:

### Calculate CW-SSIM for GT vs pix2pix, GT vs I2SB:
### from repo: https://github.com/jterrace/pyssim/tree/master

In [1]:
from ssim import SSIM
from PIL import Image, ImageOps
import os
from natsort import natsorted
import numpy as np
from tqdm import tqdm

In [2]:
GT_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\val\HE"
# p2p_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\pix2pix_sample1_only\val"
i2sb_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\i2sb\unconditional\test-run-4\iter_24388"
GT_large_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1\val\HE"
p_p2p_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1\infer\pyramid-pix2pix"
i2sb_cond_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\i2sb\conditional\test-run-seg-map-cond\iter_19656"

GT_img_list = [os.path.join(GT_img_pth,x) for x in os.listdir(GT_img_pth) if x.endswith(".png")]
# p2p_img_list = [os.path.join(p2p_img_pth,x) for x in os.listdir(p2p_img_pth) if x.endswith(".png")]
p_p2p_img_list = [os.path.join(p_p2p_img_pth,x) for x in os.listdir(p_p2p_img_pth) if x.endswith(".png")]
GT_large_img_list = [os.path.join(GT_large_img_pth,x) for x in os.listdir(GT_large_img_pth) if x.endswith(".png")]
i2sb_img_list = [os.path.join(i2sb_img_pth,x) for x in os.listdir(i2sb_img_pth) if x.endswith(".png")]
i2sb_cond_img_list = [os.path.join(i2sb_cond_img_pth,x) for x in os.listdir(i2sb_cond_img_pth) if x.endswith(".png")]

GT_img_list = natsorted(GT_img_list)
GT_large_img_list = natsorted(GT_large_img_list)
p_p2p_img_list = natsorted(p_p2p_img_list)
# p2p_img_list = natsorted(p2p_img_list)
i2sb_img_list = natsorted(i2sb_img_list)
i2sb_cond_img_list = natsorted(i2sb_cond_img_list)

### Pix2Pix CWSSIM score:

In [3]:
pix2pix_cwssim_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(p2p_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    pix2pix_cwssim_total += SSIM(ref_img).cw_ssim_value(test_img)
print("Average CWSSIM score for Pix2Pix is {}".format(pix2pix_cwssim_total / len(GT_img_list)))

### I2SB CWSSIM score:

In [10]:
i2sb_cwssim_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(i2sb_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    i2sb_cwssim_total += SSIM(ref_img).cw_ssim_value(test_img)
print("Average CWSSIM score for I2SB is {}".format(i2sb_cwssim_total / len(GT_img_list)))

Images Processed: 100%|██████████| 9951/9951 [28:33<00:00,  5.81it/s]

Average CWSSIM score for I2SB is 0.5173340810934335


### P-P2P CWSSIM Score:

In [17]:
p_p2p_cwssim_total = 0
for idx, images in tqdm(enumerate(GT_large_img_list),colour='red',desc='Images Processed', total = len(GT_large_img_list)):
    ref_img = Image.open(GT_large_img_list[idx])
    test_img = Image.open(p_p2p_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    p_p2p_cwssim_total += SSIM(ref_img).cw_ssim_value(test_img)
print("Average CWSSIM score for P-P2P is {}".format(p_p2p_cwssim_total / len(GT_large_img_list)))

Images Processed: 100%|██████████| 622/622 [25:26<00:00,  2.45s/it]

Average CWSSIM score for P-P2P is 0.5167909961986256


### I2SB-Cond CWSSIM Score:

In [20]:
i2sb_cond_cwssim_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(i2sb_cond_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    i2sb_cond_cwssim_total += SSIM(ref_img).cw_ssim_value(test_img)
print("Average CWSSIM score for I2SB-cond is {}".format(i2sb_cond_cwssim_total / len(GT_img_list)))

Images Processed: 100%|██████████| 9951/9951 [30:39<00:00,  5.41it/s]

Average CWSSIM score for I2SB-cond is 0.5937733184099259


# Calculate PSNR:

In [22]:
def calculate_psnr(img1, img2, max_value=255):
    mse = np.mean((np.array(img1, dtype=np.float32) - np.array(img2, dtype=np.float32)) ** 2)
    if mse == 0:
        return 100
    return 20 * np.log10(max_value / (np.sqrt(mse)))

### Pix2Pix PSNR Score:

In [14]:
pix2pix_psnr_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(p2p_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    pix2pix_psnr_total += calculate_psnr(ref_img, test_img)
print("Average PSNR score for Pix2Pix is {}".format(pix2pix_psnr_total / len(GT_img_list)))

Images Processed: 100%|██████████| 5952/5952 [02:36<00:00, 37.92it/s]

Average PSNR score for Pix2Pix is 14.566912677808372


### P-P2P PSNR Score:

In [37]:
p_pix2pix_psnr_total = 0
for idx, images in tqdm(enumerate(GT_large_img_list),colour='red',desc='Images Processed', total = len(GT_large_img_list)):
    ref_img = Image.open(GT_large_img_list[idx])
    test_img = Image.open(p_p2p_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    p_pix2pix_psnr_total += calculate_psnr(ref_img, test_img)
print("Average PSNR score for P-P2P is {}".format(p_pix2pix_psnr_total / len(GT_large_img_list)))

Images Processed: 100%|██████████| 622/622 [00:56<00:00, 11.06it/s]

Average PSNR score for P-P2P is 13.818377851457313


### I2SB PSNR Score:

In [43]:
i2sb_psnr_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(i2sb_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    i2sb_psnr_total += calculate_psnr(ref_img, test_img)
print("Average PSNR score for I2SB is {}".format(i2sb_psnr_total / len(GT_img_list)))

Images Processed: 100%|██████████| 9951/9951 [05:10<00:00, 32.03it/s]

Average PSNR score for I2SB is 14.843597434598804


### I2SB-cond PSNR Score:

In [23]:
i2sb_cond_psnr_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(i2sb_cond_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    i2sb_cond_psnr_total += calculate_psnr(ref_img, test_img)
print("Average PSNR score for I2SB-cond is {}".format(i2sb_cond_psnr_total / len(GT_img_list)))

Images Processed: 100%|██████████| 9951/9951 [05:51<00:00, 28.27it/s]

Average PSNR score for I2SB-cond is 16.17644426702452


# Calculate FID:
### from repo: https://github.com/mseitzer/pytorch-fid

### Pix2Pix FID Score:

In [18]:
!python -m pytorch_fid \\shelter\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\tmp\GT \\shelter\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\tmp\pix2pix_sample1_only --device cuda:0

FID:  25.728143461570426



100%|##########| 120/120 [00:43<00:00,  2.78it/s]

100%|##########| 120/120 [00:26<00:00,  4.49it/s]


### I2SB FID Score

In [1]:
!python -m pytorch_fid \\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\i2sb\unconditional\test-run-4\iter_24388 \\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\val\HE --device cuda:0

FID:  5.394903315294073


100%|##########| 200/200 [00:50<00:00,  3.96it/s]

100%|##########| 200/200 [00:38<00:00,  5.19it/s]


### P-P2P FID Score


In [2]:
!python -m pytorch_fid \\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1\infer\pyramid-pix2pix \\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1\val\HE --device cuda:0

FID:  36.64825444935218



100%|##########| 13/13 [00:25<00:00,  1.99s/it]

100%|##########| 13/13 [00:19<00:00,  1.50s/it]


### I2SB-cond FID Score

In [24]:
!python -m pytorch_fid \\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\i2sb\conditional\test-run-seg-map-cond\iter_19656 \\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\val\HE --device cuda:0

FID:  5.104889101952949



100%|##########| 200/200 [01:06<00:00,  3.00it/s]

100%|##########| 200/200 [01:02<00:00,  3.19it/s]


### Trying DISTS (lower the better), LPIPS (lower the better), style loss (lower the better), VSI (0-1, higher the better), and haarpsi (0-1, higher the better) as a metric:

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from piq import DISTS, LPIPS, vsi, haarpsi, StyleLoss
from torchvision.transforms import ToTensor
import torchvision.transforms as T

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tensor_transform = T.Compose([ToTensor()]) # [0,255] -> [0,1], HWC -> CHW

### For Pix2Pix (Batch Size doesn't change the results, higher batch size for faster computation):

In [5]:
class TestDataset(Dataset):
    def __init__(self, GT_image_list, test_img_list, transforms=None):
        self.GT_image_list = GT_image_list
        self.test_img_list = test_img_list
        self.transforms = transforms

    def __len__(self):
        if len(self.GT_image_list) != len(self.test_img_list):
            assert("Length of Test and GT Image List Unequal")
        return len(self.GT_image_list)

    def __getitem__(self,idx):
        ref_img = np.array(Image.open(self.GT_image_list[idx]))
        test_img = np.array(Image.open(self.test_img_list[idx]))
        if self.transforms is not None:
            ref_img = self.transforms(ref_img)
            test_img = self.transforms(test_img)
        return ref_img, test_img

In [12]:
p2p_img_dataset = TestDataset(GT_img_list= GT_img_list, test_img_list= p2p_img_list, transforms = tensor_transform)
p2p_img_dataloader = DataLoader(dataset=p2p_img_dataset,
                                  batch_size=32,
                                  #pin_memory= true allows faster data transport from cpu to gpu
                                  num_workers=0, pin_memory=True, shuffle=False)
#check
ref_img_batch, p2p_img_batch = next(iter(p2p_img_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(ref_img_batch.size(), p2p_img_batch.size()))

Images have a tensor size of torch.Size([32, 3, 256, 256]), and Labels have a tensor size of torch.Size([32, 3, 256, 256])


In [13]:
pix2pix_dists_total = 0
pix2pix_lpips_total = 0
pix2pix_vsi_total = 0
pix2pix_haarpsi_total = 0
pix2pix_style_loss_total = 0
pbar = tqdm(enumerate(p2p_img_dataloader), total=len(p2p_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, p2p_img) in pbar:
    ref_img = ref_img.to(device)
    p2p_img = p2p_img.to(device)
    pix2pix_vsi_total += vsi(p2p_img,ref_img)
    pix2pix_dists_total += DISTS()(p2p_img,ref_img)
    pix2pix_lpips_total += LPIPS()(p2p_img,ref_img)
    pix2pix_haarpsi_total += haarpsi(p2p_img,ref_img)
    pix2pix_style_loss_total += StyleLoss()(p2p_img,ref_img)
print("Average VSI score for Pix2Pix is {}".format(pix2pix_vsi_total / len(p2p_img_dataloader)))
print("Average DISTS score for Pix2Pix is {}".format(pix2pix_dists_total / len(p2p_img_dataloader)))
print("Average LPIPS score for Pix2Pix is {}".format(pix2pix_lpips_total / len(p2p_img_dataloader)))
print("Average HAARPSI score for Pix2Pix is {}".format(pix2pix_haarpsi_total / len(p2p_img_dataloader)))
print("Average Style score for Pix2Pix is {}".format(pix2pix_style_loss_total / len(p2p_img_dataloader)))

Images Processed: 100%|██████████| 186/186 [11:24<00:00,  3.68s/it]

Average VSI score for Pix2Pix is 0.834528923034668
Average DISTS score for Pix2Pix is 0.2199782282114029
Average LPIPS score for Pix2Pix is 0.4710122048854828
Average HAARPSI score for Pix2Pix is 0.35326284170150757
Average Style score for Pix2Pix is 20185016.0


## For P-P2P:

In [11]:
len(GT_large_img_list)

622

In [10]:
p_p2p_img_dataset = TestDataset(GT_image_list= GT_large_img_list, test_img_list= p_p2p_img_list, transforms = tensor_transform)
p_p2p_img_dataloader = DataLoader(dataset=p_p2p_img_dataset,
                                  batch_size=4,
                                  #pin_memory= true allows faster data transport from cpu to gpu
                                  num_workers=0, pin_memory=True, shuffle=False)
#check
ref_img_batch, p_p2p_img_batch = next(iter(p_p2p_img_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(ref_img_batch.size(), p_p2p_img_batch.size()))

Images have a tensor size of torch.Size([4, 3, 1024, 1024]), and Labels have a tensor size of torch.Size([4, 3, 1024, 1024])


In [ ]:
p_pix2pix_dists_total = 0
p_pix2pix_lpips_total = 0
p_pix2pix_vsi_total = 0
p_pix2pix_haarpsi_total = 0
p_pix2pix_style_loss_total = 0
pbar = tqdm(enumerate(p_p2p_img_dataloader), total=len(p_p2p_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, p_p2p_img) in pbar:
    ref_img = ref_img.to(device)
    p_p2p_img = p_p2p_img.to(device)
    p_pix2pix_vsi_total += vsi(p_p2p_img,ref_img)
    p_pix2pix_dists_total += DISTS()(p_p2p_img,ref_img)
    p_pix2pix_lpips_total += LPIPS()(p_p2p_img,ref_img)
    p_pix2pix_haarpsi_total += haarpsi(p_p2p_img,ref_img)
    p_pix2pix_style_loss_total += StyleLoss()(p_p2p_img,ref_img)
print("Average VSI score for Pix2Pix is {}".format(p_pix2pix_vsi_total / len(p_p2p_img_dataloader)))
print("Average DISTS score for Pix2Pix is {}".format(p_pix2pix_dists_total / len(p_p2p_img_dataloader)))
print("Average LPIPS score for Pix2Pix is {}".format(p_pix2pix_lpips_total / len(p_p2p_img_dataloader)))
print("Average HAARPSI score for Pix2Pix is {}".format(p_pix2pix_haarpsi_total / len(p_p2p_img_dataloader)))
print("Average Style score for Pix2Pix is {}".format(p_pix2pix_style_loss_total / len(p_p2p_img_dataloader)))

### For I2SB:

In [6]:
i2sb_img_dataset = TestDataset(GT_image_list= GT_img_list, test_img_list= i2sb_img_list, transforms = tensor_transform)
i2sb_img_dataloader = DataLoader(dataset=i2sb_img_dataset,
                                  batch_size=32,
                                  #pin_memory= true allows faster data transport from cpu to gpu
                                  num_workers=0, pin_memory=True, shuffle=False)
#check:
ref_img_batch, i2sb_img_batch = next(iter(i2sb_img_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(ref_img_batch.size(), i2sb_img_batch.size()))

Images have a tensor size of torch.Size([32, 3, 256, 256]), and Labels have a tensor size of torch.Size([32, 3, 256, 256])


In [33]:
i2sb_dists_total = 0
i2sb_lpips_total = 0
i2sb_vsi_total = 0
i2sb_haarpsi_total = 0
i2sb_style_loss_total = 0
pbar = tqdm(enumerate(i2sb_img_dataloader), total=len(i2sb_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, i2sb_img) in pbar:
    ref_img = ref_img.to(device)
    i2sb_img = i2sb_img.to(device)
    i2sb_vsi_total += vsi(i2sb_img,ref_img)
    i2sb_dists_total += DISTS()(i2sb_img,ref_img)
    i2sb_lpips_total += LPIPS()(i2sb_img,ref_img)
    i2sb_haarpsi_total += haarpsi(i2sb_img,ref_img)
    i2sb_style_loss_total += StyleLoss()(i2sb_img,ref_img)
print("Average VSI score for I2SB is {}".format(i2sb_vsi_total / len(i2sb_img_dataloader)))
print("Average DISTS score for I2SB is {}".format(i2sb_dists_total / len(i2sb_img_dataloader)))
print("Average LPIPS score for I2SB is {}".format(i2sb_lpips_total / len(i2sb_img_dataloader)))
print("Average HAARPSI score for I2SB is {}".format(i2sb_haarpsi_total / len(i2sb_img_dataloader)))
print("Average Style score for I2SB is {}".format(i2sb_style_loss_total / len(i2sb_img_dataloader)))

Images Processed:   0%|          | 0/311 [00:00<?, ?it/s]C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Images Processed: 100%|██████████| 311/311 [19:22<00:00,  3.74s/it]

Average VSI score for I2SB is 0.8375942707061768
Average DISTS score for I2SB is 0.18147222697734833
Average LPIPS score for I2SB is 0.4420967698097229
Average HAARPSI score for I2SB is 0.36006778478622437
Average Style score for I2SB is 7222112.0


### For I2SB-cond:

In [7]:
i2sb_cond_img_dataset = TestDataset(GT_image_list= GT_img_list, test_img_list= i2sb_cond_img_list, transforms = tensor_transform)
i2sb_cond_img_dataloader = DataLoader(dataset=i2sb_cond_img_dataset,
                                  batch_size=32,
                                  #pin_memory= true allows faster data transport from cpu to gpu
                                  num_workers=0, pin_memory=True, shuffle=False)
#check:
ref_img_batch, i2sb_cond_img_batch = next(iter(i2sb_cond_img_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(ref_img_batch.size(), i2sb_cond_img_batch.size()))

Images have a tensor size of torch.Size([32, 3, 256, 256]), and Labels have a tensor size of torch.Size([32, 3, 256, 256])


In [29]:
i2sb_cond_dists_total = 0
i2sb_cond_lpips_total = 0
i2sb_cond_vsi_total = 0
i2sb_cond_haarpsi_total = 0
i2sb_cond_style_loss_total = 0
pbar = tqdm(enumerate(i2sb_cond_img_dataloader), total=len(i2sb_cond_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, i2sb_cond_img) in pbar:
    ref_img = ref_img.to(device)
    i2sb_cond_img = i2sb_cond_img.to(device)
    i2sb_cond_vsi_total += vsi(i2sb_cond_img,ref_img)
    i2sb_cond_dists_total += DISTS()(i2sb_cond_img,ref_img)
    i2sb_cond_lpips_total += LPIPS()(i2sb_cond_img,ref_img)
    i2sb_cond_haarpsi_total += haarpsi(i2sb_cond_img,ref_img)
    i2sb_cond_style_loss_total += StyleLoss()(i2sb_cond_img,ref_img)
print("Average VSI score for I2SB-cond is {}".format(i2sb_cond_vsi_total / len(i2sb_cond_img_dataloader)))
print("Average DISTS score for I2SB-cond is {}".format(i2sb_cond_dists_total / len(i2sb_cond_img_dataloader)))
print("Average LPIPS score for I2SB-cond is {}".format(i2sb_cond_lpips_total / len(i2sb_cond_img_dataloader)))
print("Average HAARPSI score for I2SB-cond is {}".format(i2sb_cond_haarpsi_total / len(i2sb_cond_img_dataloader)))
print("Average Style score for I2SB-cond is {}".format(i2sb_cond_style_loss_total / len(i2sb_cond_img_dataloader)))

Images Processed:   0%|          | 0/311 [00:00<?, ?it/s]C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Images Processed: 100%|██████████| 311/311 [1:18:23<00:00, 15.12s/it]

Average VSI score for I2SB-cond is 0.8820433616638184
Average DISTS score for I2SB-cond is 0.16060639917850494
Average LPIPS score for I2SB-cond is 0.3861348330974579
Average HAARPSI score for I2SB-cond is 0.4720633625984192
Average Style score for I2SB-cond is 6289622.5


# SSIM and PieAPP Score:

In [8]:
from piq import PieAPP, ssim

### For Pix2Pix:

In [30]:
pix2pix_pieapp_total = 0
pix2pix_ssim_total = 0
pbar = tqdm(enumerate(p2p_img_dataloader), total=len(p2p_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, p2p_img) in pbar:
    ref_img = ref_img.to(device)
    p2p_img = p2p_img.to(device)
    pix2pix_pieapp_total += PieAPP()(p2p_img,ref_img)
    pix2pix_ssim_total += ssim(p2p_img,ref_img)
print("Average PieAPP score for Pix2Pix is {}".format(pix2pix_pieapp_total / len(p2p_img_dataloader)))
print("Average SSIM score for Pix2Pix is {}".format(pix2pix_ssim_total / len(p2p_img_dataloader)))


Images Processed: 100%|██████████| 186/186 [22:16<00:00,  7.19s/it]

Average PieAPP score for Pix2Pix is 2.0399372577667236
Average SSIM score for Pix2Pix is 0.18936103582382202


### P-Pix2Pix SSIM and PieAPP Score:

In [14]:
p_pix2pix_pieapp_total = 0
p_pix2pix_ssim_total = 0
pbar = tqdm(enumerate(p_p2p_img_dataloader), total=len(p_p2p_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, i2sb_img) in pbar:
    ref_img = ref_img.to(device)
    i2sb_img = i2sb_img.to(device)
    p_pix2pix_pieapp_total += PieAPP()(i2sb_img,ref_img)
    p_pix2pix_ssim_total += ssim(i2sb_img,ref_img)
print("Average PieAPP score for P-P2P is {}".format(p_pix2pix_pieapp_total / len(p_p2p_img_dataloader)))
print("Average SSIM score for P-P2P is {}".format(p_pix2pix_ssim_total / len(p_p2p_img_dataloader)))


Images Processed: 100%|██████████| 156/156 [52:14<00:00, 20.09s/it]

Average PieAPP score for P-P2P is 1.5467543601989746
Average SSIM score for P-P2P is 0.24277183413505554


### I2SB SSIM and PieAPP Score:


In [46]:
i2sb_pieapp_total = 0
i2sb_ssim_total = 0
pbar = tqdm(enumerate(i2sb_img_dataloader), total=len(i2sb_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, i2sb_img) in pbar:
    ref_img = ref_img.to(device)
    i2sb_img = i2sb_img.to(device)
    i2sb_pieapp_total += PieAPP()(i2sb_img,ref_img)
    i2sb_ssim_total += ssim(i2sb_img,ref_img)
print("Average PieAPP score for I2SB is {}".format(i2sb_pieapp_total / len(i2sb_img_dataloader)))
print("Average SSIM score for I2SB is {}".format(i2sb_ssim_total / len(i2sb_img_dataloader)))


Images Processed: 100%|██████████| 311/311 [44:33<00:00,  8.60s/it]

Average PieAPP score for I2SB is 1.5017690658569336
Average SSIM score for I2SB is 0.2087206244468689


### I2SB-cond SSIM and PieAPP Score:

In [9]:
i2sb_cond_pieapp_total = 0
i2sb_cond_ssim_total = 0
pbar = tqdm(enumerate(i2sb_cond_img_dataloader), total=len(i2sb_cond_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, i2sb_cond_img) in pbar:
    ref_img = ref_img.to(device)
    i2sb_cond_img = i2sb_cond_img.to(device)
    i2sb_cond_pieapp_total += PieAPP()(i2sb_cond_img,ref_img)
    i2sb_cond_ssim_total += ssim(i2sb_cond_img,ref_img)
print("Average PieAPP score for I2SB is {}".format(i2sb_cond_pieapp_total / len(i2sb_cond_img_dataloader)))
print("Average SSIM score for I2SB is {}".format(i2sb_cond_ssim_total / len(i2sb_cond_img_dataloader)))


Images Processed: 100%|██████████| 311/311 [10:15<00:00,  1.98s/it]

Average PieAPP score for I2SB is 1.325861930847168
Average SSIM score for I2SB is 0.2672995328903198


### Calculate Haralick features:

In [ ]:
import os
import cv2
import mahotas
import numpy as np
import pandas as pd
import csv


In [ ]:
def calculate_haralick(image_path):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Calculate Haralick features
    haralick_features = mahotas.features.haralick(image).mean(axis=0)

    return haralick_features


def process_images_in_folder(folder_path, output_csv):
    # Get a list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.tif', '.jpg', '.png'))]

    # Write header to CSV file
    with open(output_csv, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        header = ['Image'] + [f'Haralick_{i}' for i in range(13)]  # 13 Haralick features
        writer.writerow(header)

        # Process each image and write results to CSV
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            haralick_features = calculate_haralick(image_path)

            # Write results to CSV
            row = [image_file] + list(haralick_features)
            writer.writerow(row)

In [ ]:
input_folder = r'\\10.99.68.178\Saurabh\Saurabh\Pancreas_Ashley_Files\new versions\5x\registered\Duct_1\linear\Skip_7'
output_csv = 'output_haralick_features1.csv'

process_images_in_folder(input_folder, output_csv)
print(f"Haralick features have been computed and saved to {output_csv}.")